# Train Translator

Based on: https://huggingface.co/docs/transformers/tasks/translation

In [1]:
import sys, os, datetime
import json
import torch
import random
import glob
from tqdm.notebook import tqdm
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import TranslationPipeline
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets

2023-05-31 13:45:25.782634: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 13:45:25.896777: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-31 13:45:26.289123: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:
2023-05-31 13:45:26.289171: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [2]:
import cdli
import languages

In [3]:
def get_finetune_model_id(model_id):
    model_dir = f"../results/{model_id}"
    checkpoints = [(os.path.abspath(x), int(os.path.split(x)[1].split("-")[1])) for x in glob.glob(f"{model_dir}/checkpoint-*")]
    checkpoints = sorted(checkpoints, key=lambda x: x[1])[-1]
    return checkpoints[0]

In [4]:
os.environ["WANDB_NOTEBOOK_NAME"] = "TrainTranslatorNew.ipynb"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

source_langs = set(["akk", "sux"])

# target_langs = set(["en", "it", "es", "fr", "de"])
target_langs = set(["en"])

base_model_id = "t5-base"
finetune_model_id = None
# finetune_model_id = get_finetune_model_id("t5-base-p-akksux-en-20220722-173018")

model_max_length = 512
batch_size = 4 if os.path.basename(base_model_id).startswith("t5-large") else (8 if os.path.basename(base_model_id).startswith("t5-base") else 128)

num_train_epochs = 30

is_bi = True
is_finetune = finetune_model_id is not None and len(finetune_model_id) > 1

In [5]:
date_id = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
flags = ""
suffix = ""
if is_bi:
    flags += "-bi"
if is_finetune:
    flags += "-f"
    suffix += f"-{os.path.basename(os.path.split(finetune_model_id)[0])}-{os.path.basename(finetune_model_id)}"
model_id = f"{os.path.basename(base_model_id)}{flags}-{''.join(sorted(list(source_langs)))}-{''.join(sorted(list(target_langs)))}-{date_id}{suffix}"
model_id

't5-base-bi-akksux-en-20230531-134528'

In [6]:
has_cuda = torch.cuda.is_available()
device = torch.cuda.device(0) if has_cuda else "cpu"
has_cuda, device

(True, <torch.cuda.device at 0x7f29b9da5930>)

In [7]:
!nvidia-smi

Wed May 31 13:45:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
| 30%   44C    P8    32W / 350W |    103MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load Training Data

In [8]:
def get_prefix(src_lang, tgt_lang):
    s = languages.all_languages[src_lang]
    t = languages.all_languages[tgt_lang]
    return f"translate {s} to {t}: "
    
get_prefix("suxts", "es")

'translate Sumerian to Spanish: '

In [9]:
def load_translations_dataset(from_lang, to_lang):
    ds = Dataset.from_json(f"../data/translations_{from_lang}_to_{to_lang}.jsonl")
    srcs = [get_prefix(from_lang, to_lang) + x[from_lang] for x in ds]
    targets = [x[to_lang] for x in ds]
    if is_bi:
        srcs.extend([get_prefix(to_lang, from_lang) + x[to_lang] for x in ds])
        targets.extend([x[from_lang] for x in ds])
#     ds = ds.add_column("source", srcs)
#     ds = ds.add_column("target", targets)
#     ds = ds.remove_columns([from_lang])
#     ds = ds.rename_column(to_lang, "target")
    ds = Dataset.from_dict({"source": srcs, "target": targets})
    return ds
    
translation_datasets = {lang: load_translations_dataset(lang, "en") for lang in source_langs}

Using custom data configuration default-0536cdbcc7e585ea
Reusing dataset json (/home/fak/.cache/huggingface/datasets/json/default-0536cdbcc7e585ea/0.0.0)
Using custom data configuration default-e1c5548be563d45e
Reusing dataset json (/home/fak/.cache/huggingface/datasets/json/default-e1c5548be563d45e/0.0.0)


In [10]:
translation_datasets

{'akk': Dataset({
     features: ['source', 'target'],
     num_rows: 243672
 }),
 'sux': Dataset({
     features: ['source', 'target'],
     num_rows: 119400
 })}

In [11]:
translation_datasets["sux"][1]

{'source': 'translate Sumerian to English: # = %a ru-ub-bu-u2 # e3 ~ = %a e-ru-u2-um # e3 ~ = %a a-ma-<<x>>-rum',
 'target': 'to bring up ? to watch'}

In [12]:
merged_dataset = concatenate_datasets(translation_datasets.values())

In [13]:
merged_dataset

Dataset({
    features: ['source', 'target'],
    num_rows: 363072
})

In [14]:
dataset = merged_dataset.shuffle()

In [15]:
dataset[1]

{'source': 'translate English to Akkadian: Witnesses: Marduk-nāṣir, ērib-bīti of Nabû, son of Nabû-tabni-uṣur of the Nūr-Papsukkal family, Šulā/Rēmūt/Ea-ilūtu-bani, Arad-Tašmētu/Nabû-nādin-ahi/Huṣābu, Nabû-ahhe-bulliṭ/Nabû-nādin-ahi/Huṣābu.',
 'target': '_dumu_-szu2 sza2 {disz}{d}_ag_-_du3_-_uru3 a_ {disz}_zalag2_-{d}_pap_-[_sukkal_] [{disz}szu]-la-a A-szu2 sza2 {disz}re-mut A {disz}{d}_be_-_dingir_-ta-[_du3_] [{disz}_arad2_]-{d}tasz-me-tum A-szu2 sza2 {disz}{d}_ag_-na-din-_szesz a_ {disz}hu-s,a-[bi] {disz}{d}_ag_-_szesz_-_mesz_-_tin a_-szu2 sza2 {disz}{d}_ag_-na-din-_szesz a_ {disz}hu-s,a-[bi] u3 {lu2}_dub-sar_ {disz}{d}_utu_-_mu_-_si-sa2 a_-szu2 sza2 {disz}{d}_ag_-_mu_-_uru3_{?}'}

In [16]:
avg_src_chars_per_token = 1.8338974021110785
avg_tgt_chars_per_token = 2.829482016086902

In [17]:
def target_ok(target_text):
    if len(target_text) == 0:
        return False
    if len(set(target_text.replace(" ", ""))) < 2:
        return False
    return True
    

def test_target_ok(text):
    ok = target_ok(text)
    print(ok, repr(text))
    
test_target_ok("")
test_target_ok(" ")
test_target_ok("xx xxx x")
test_target_ok(".. . .. ")
test_target_ok("Hi")

False ''
False ' '
False 'xx xxx x'
False '.. . .. '
True 'Hi'


In [18]:
translations = dataset.train_test_split(test_size=0.1)

In [19]:
train_dataset, test_dataset = translations["train"], translations["test"]

In [20]:
test_dataset

Dataset({
    features: ['source', 'target'],
    num_rows: 36308
})

In [21]:
test_dataset[1120:1200]

{'source': ['translate English to Sumerian: have set me before the god, my creator, in a place where there is life and peace for me.',
  'translate Akkadian to English: 14 4-30 _me_',
  'translate Akkadian to English: [ká-hi-li-sù]',
  'translate Sumerian to English: [... szul]-mu _min_ ba-asz-tum',
  'translate English to Akkadian: Witness Urdu-Nanaya, priest of Šamaš.',
  'translate Sumerian to English: mu gu-za',
  'translate English to Akkadian: Have they perhaps said to you on this matter: "Your father enthroned his substitutes there." These words are rubbish! Why did the king not say to them like this: "The evil of my father was in one region, mine is in another; the evil of Assyria and Babylonia are not interrelated: when a sign pertaining to Assyria appears, the ritual should be performed here, and when a sign pertaining to Babylonia appears, it should be done there." Now the king of Babylonia is well ...... You are still ......',
  'translate English to Akkadian: ... ... Month

## Tokenize the Data

In [22]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, model_max_length=model_max_length)

In [23]:
tokenizer.model_max_length

512

In [24]:
print("pad", tokenizer.pad_token, tokenizer.pad_token_id)
print("eos", tokenizer.eos_token, tokenizer.eos_token_id)
print("unk", tokenizer.unk_token, tokenizer.unk_token_id)

pad <pad> 0
eos </s> 1
unk <unk> 2


In [25]:
ccc = 0
sum_src_chars_per_token = 0.0
num_src_chars_per_token = 0
sum_tgt_chars_per_token = 0.0
num_tgt_chars_per_token = 0

def preprocess_function(examples):
    global ccc, sum_src_chars_per_token, sum_tgt_chars_per_token, num_src_chars_per_token, num_tgt_chars_per_token
#     print(examples)
    inputs = [example for example in examples["source"]]
    targets = [example for example in examples["target"]]
    model_inputs = tokenizer(inputs, max_length=model_max_length, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=model_max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    
    nexamples = len(inputs)
    for i in range(nexamples):
        nchar = len(inputs[i])
        ntoks = len(model_inputs["input_ids"][i])
        if ntoks > 0:
            sum_src_chars_per_token += nchar / ntoks
            num_src_chars_per_token += 1
        nchar = len(targets[i])
        ntoks = len(model_inputs["labels"][i])
        if ntoks > 0:
            sum_tgt_chars_per_token += nchar / ntoks
            num_tgt_chars_per_token += 1
    
    ccc += 1
    if ccc == 1:
        print(model_inputs["input_ids"][0])
        print(model_inputs["labels"][0])
        nchar = len(targets[0])
        ntoks = len(model_inputs["labels"][0])
        print(nchar, ntoks, nchar / ntoks)

    return model_inputs

tokenized_translations = translations.map(preprocess_function, batched=True)
tokenized_translations

Parameter 'function'=<function preprocess_function at 0x7f29e91e7400> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/327 [00:00<?, ?ba/s]

[13959, 4823, 1258, 8603, 12, 1566, 10, 784, 1935, 3, 2, 4115, 2, 26, 173, 18, 3697, 3, 233, 908, 3, 226, 4663, 3, 23, 18, 13363, 18, 3007, 18, 76, 17, 6, 1]
[105, 5801, 22301, 3, 233, 9567, 7, 1]
18 8 2.25


  0%|          | 0/37 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 326764
    })
    test: Dataset({
        features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 36308
    })
})

In [26]:
avg_src_chars_per_token = sum_src_chars_per_token / num_src_chars_per_token
avg_tgt_chars_per_token = sum_tgt_chars_per_token / num_tgt_chars_per_token
print("avg_src_chars_per_token", "=", avg_src_chars_per_token)
print("avg_tgt_chars_per_token", "=", avg_tgt_chars_per_token)

avg_src_chars_per_token = 2.5855966941228634
avg_tgt_chars_per_token = 2.0561622926504537


In [27]:
tokenized_translations["train"] = tokenized_translations["train"].remove_columns(["source", "target"])
tokenized_translations["test"] = tokenized_translations["test"].remove_columns(["source", "target"])
tokenized_translations

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 326764
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 36308
    })
})

In [28]:
tokenized_translations["train"][0]["labels"][:10]

[105, 5801, 22301, 3, 233, 9567, 7, 1]

## Load the Model

In [29]:
model = AutoModelForSeq2SeqLM.from_pretrained(finetune_model_id if is_finetune else base_model_id, 
                                              max_length=model_max_length)

In [30]:
model.config

T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "max_length": 512,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beams": 4,
      "prefix": "translate English to Ge

## Train

In [31]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
# data_collator

In [32]:
training_args = Seq2SeqTrainingArguments(
    output_dir=f"../results/{model_id}",
    evaluation_strategy="epoch",
    learning_rate=2.0e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    fp16=has_cuda,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_translations["train"],
    eval_dataset=tokenized_translations["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)



Using amp half precision backend


In [ ]:
trainer.train()

/home/fak/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 326764
  Num Epochs = 30
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1225380
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: praeclarum. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,1.816700,1.604921
2,1.563000,1.379398
3,1.400400,1.257212
4,1.325900,1.177363
5,1.248400,1.119924
6,1.188400,1.073896
7,1.134900,1.038351
8,1.096400,1.009079
9,1.051000,0.983964
10,1.045700,0.962791


Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-500
Configuration saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-1000
Configuration saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-1000/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-202

Model weights saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-7000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-7000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-7000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-5500] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-7500
Configuration saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-7500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-7500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-7500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-7500/special_tokens_map.json
Deleting ol

Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-13500
Configuration saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-13500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-13500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-13500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-13500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-12000] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-14000
Configuration saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-14000/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-14000/pytorch_model.bin
tokenizer config file saved in ../results/t5-b

...

Configuration saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-600000/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-600000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-600000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-600000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-598500] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-600500
Configuration saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-600500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-600500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-600500/tokenizer_config.json
Special tok

Special tokens file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-606000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-604500] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-606500
Configuration saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-606500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-606500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-606500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-606500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-605000] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-607000
Con

Configuration saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-612500/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-612500/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-612500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-612500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-611000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 36308
  Batch size = 8
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-613000
Configuration saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-613000/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-613000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux

tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-618500/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-618500/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-617000] due to args.save_total_limit
Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-619000
Configuration saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-619000/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-619000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-619000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-619000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-617500] due to

Saving model checkpoint to ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-625000
Configuration saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-625000/config.json
Model weights saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-625000/pytorch_model.bin
tokenizer config file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-625000/tokenizer_config.json
Special tokens file saved in ../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-625000/special_tokens_map.json
Deleting older checkpoint [../results/t5-base-bi-akksux-en-20230531-134528/checkpoint-623500] due to args.save_total_limit


## Sample

In [ ]:
pipeline = TranslationPipeline(model=model.to("cpu"), tokenizer=tokenizer, max_length=model_max_length)

In [ ]:
pipeline

In [ ]:
pipeline("translate English to French: hello my name is Frank")

In [ ]:
source_test = translations["test"][0]["source"]
target_test = translations["test"][0]["target"]
print(source_test)
print("-"*80)
print(target_test)

In [ ]:
def translate(text):
    return pipeline(text)

translate(source_test)

In [ ]:
tests = original_tests
def sample(num_samples=100):
    for i in range(min(num_samples, tests.num_rows)):
        t = tests[i]
    #     print(t)
        src = t["source"]
        tgt = t["target"]
        query = src
        pred = pipeline(query)[0]["translation_text"]
        print("-"*48)
        print("QUERY ", query)
        print("TARGET", tgt)
        print("PRED  ", pred)
    #     break
    
sample()

## Save to Huggingface

In [ ]:
model_path = os.path.abspath(f"/home/fak/nn/Data/generated/cuneiform/{model_id}-fullytrained")
trainer.save_model(model_path)
model_path

In [ ]:
tokenizer.save_pretrained(model_path)